<a href="https://colab.research.google.com/github/Rithvik26/Ford_dataset_nbc/blob/main/Ford_Classification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from collections import Counter
import string
import random
import math
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

from sklearn.metrics import accuracy_score

import math

import nltk

from sklearn.feature_extraction.text import CountVectorizer

from collections import defaultdict


In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/Rithvik26/Ford_dataset_nbc/main/train_data.csv')
print(data.head())

   Unnamed: 0  Sentence_id                                       New_Sentence  \
0           0  GERRES15609  Author and/or Review architecture/design and o...   
1           1  PHERES15784  Should be able to develop custom dynamic shape...   
2           2  GERREQ10457  Experience in working crosslly with a  larger ...   
3           3  GERSKL27235  Previous business experience, including but no...   
4           4  HONSSK18415         Delivering fast and right the first  time.   

             Type  
0  Responsibility  
1  Responsibility  
2     Requirement  
3           Skill  
4       SoftSkill  


In [3]:
def rem_tags(st):
    r_l = ""
    if type(st)!=str:
      return ''
    result = re.sub('','',st)          #rem HTML tags
    result = re.sub('https://.*','',result)   #rem Urls
    result = re.sub(r'[^w'+r_l+']', ' ',result)    #rem non-alphanumeric characters 
    result = result.lower()
    return result
data['New_Sentence']=data['New_Sentence'].apply(lambda cw : rem_tags(cw)) 
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['New_Sentence'] = data['New_Sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import nltk
nltk.download('all')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['New_Sentence'] = data.New_Sentence.apply(lemmatize_text)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [5]:
Sentences = data['New_Sentence'].values
labels = data['Type'].values
en = LabelEncoder()
encoded_lab = en.fit_transform(labels)
print(encoded_lab)

[3 3 2 ... 5 5 0]


In [7]:
#Splitting into 6:2:2
x, test_sentences, y, test_labels = train_test_split(Sentences,encoded_lab,test_size=0.2,train_size=0.8)
train_sentences, dev_sentences, train_labels, dev_labels = train_test_split(x,y,test_size = 0.25,train_size =0.75)

In [8]:
print ("Total Number of Training Examples: ",len(train_sentences))
print ("Total Number of Training Labels: ",len(train_labels))

Total Number of Training Examples:  36069
Total Number of Training Labels:  36069


In [9]:
print ("Number of DEV Examples: ",len(dev_sentences))
print ("Number of DEV Labels: ",len(dev_labels))

Number of DEV Examples:  12023
Number of DEV Labels:  12023


In [10]:
print ("Number of Test Examples: ",len(test_sentences))
print ("Number of Test Labels: ",len(test_labels))

Number of Test Examples:  12023
Number of Test Labels:  12023


In [11]:
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names()
X = X.toarray()
word_counts = {}
def count_words(X,t_lab,voc):
  for l in range(6):
      word_counts[l] = defaultdict(lambda: 0)
  for i in range(X.shape[0]):
      l = t_lab[i]
      for j in range(len(voc)):
          word_counts[l][voc[j]] += X[i][j]
  return word_counts
word_counts = count_words(X,train_labels,vocab)

In [12]:
#My Contribution
class NaiveBayes():
  def __init__(self,l):
    self.num_sen_labels={}
    self.cond_prob={}
    self.labels = l
  def groupby_lab(self,x, y):
    q = {}
    for l in self.labels:
        q[l] = x[np.where(y == l)]
    return q
  def laplace_smoothing(self, voc, word_c, word, sent_l):
      a = word_c[sent_l][word] + 1
      b = self.num_sen_labels[sent_l] + len(voc)
      return math.log(a/b)
  def fit(self,x, y):
      g_d = self.groupby_lab(x, y)
      for i,j in g_d.items(): 
          self.num_sen_labels[i] = len(j)
          self.cond_prob[i] = math.log(self.num_sen_labels[i] / len(x))
      
  def predict(self, voc, word_c, x):
      result = []
      for text in x:
          l_s = {l: self.cond_prob[l] for l in self.labels}

          words = set(w_tokenizer.tokenize(text))
          for word in words:
              if word not in vocab: continue
              for l in self.labels:
                  word_prob = self.laplace_smoothing(vocab, word_c, word, l)
                  l_s[l] += word_prob
      
          result.append(max(l_s, key=l_s.get))
      return result

In [13]:
model=NaiveBayes([0,1,2,3,4,5])
model.fit(train_sentences,train_labels)
pred = model.predict(vocab, word_counts,dev_sentences)
print("Accuracy of prediction on development set : ", 100*accuracy_score(dev_labels,pred))

Accuracy of prediction on development set :  26.066705481161108


In [14]:
pred = model.predict(vocab, word_counts,test_sentences)
print("Final Accuracy of prediction on Test set : ", 100*accuracy_score(test_labels,pred))

Final Accuracy of prediction on Test set :  25.783914164518006
